# LLVM Primer

LLVM is the engine that drives our effort. It is a modern compiler framework and intermediate representation language together with toolchain for manipulating and optimizing this language.

## Basic Types

LLVM types are your typical machine types plus pointers, structs, vectors and arrays. In this course we do not use structs and vectors. Some examples:

```
i1                     ; boolean bit
i32                    ; integer
float                  ; single precision
[10 x float]           ; Array of 10 floats
[10 x [20 x i32]]      ; Array of 10 arrays of 20 integers.
[25 x float]*          ; Pointer to an array
```

## Instructions

All instructions are assignment to a unique virtual register. In SSA (Single Static Assignment) a register is never assigned to more than once.
```
%result = add i32 10, 20
```

Symbols used in an LLVM module are either global or local. Global symbols begin with @ and local symbols begin with %.

The numerical instructions are:

```
add : Integer addition
fadd : Floating point addition
sub : Integer subtraction
fsub : Floating point subtraction
mul : Integer multiplication
fmul : Floating point multiplication
udiv : Unsigned integer quotient
sdiv : Signed integer quotient
fdiv : Floating point quotient
urem : Unsigned integer remainder
srem : Signed integer remainder
frem : Floating point integer remainder
```

## Memory

LLVM uses the traditional load/store model:

```
load: Load a typed value from a given reference
store: Store a typed value in a given reference
alloca: Allocate a pointer to memory on the virtual stack
```
Examples:
```
%ptr = alloca i32
store i32 3, i32* %ptr
%val = load i32* %ptr
```

## Functions

Functions are defined by as a collection of basic blocks, a return type and argument types. Function names must be unique in the module.
```
define i32 @add(i32 %a, i32 %b) {
  %1 = add i32 %a, %b
  ret i32 %1
}
```

## Basic Blocks

The function is split across basic blocks which hold sequences of instructions and a terminator instruction which either returns or jumps to another local basic block.
```
define i1 @foo() {
entry:
  br label %next
next:
  br label %return
return:
  ret i1 0
}
```

## Return

A function must have a terminator, one of such instructions is a ret which returns a value to the stack.
```
define i1 @foo() {
  ret i1 0
}
```

## Unconditional Branch

An unconditional branch jumps unconditionally to a labeled basic block.
```
define i1 @foo() {
start:
  br label %next
next:
  br label %return
return:
  ret i1 0
}
```

## Conditional Branch
```
define i32 @foo() {
start:
  br i1 true, label %left, label %right
left:
  ret i32 10
right:
  ret i32 20
}
```

## Phi

Phi nodes yield a value that depends on the operand corresponding to their predecessor basic block. These are used for implementing loops in SSA.
```
define i32 @foo() {
start:
  br i1 true, label %left, label %right
left:
  %plusOne = add i32 0, 1
  br label %merge
right:
  br label %merge
merge:
  %join = phi i32 [ %plusOne, %left], [ -1, %right]
  ret i32 %join
}
```

## Loops

Loops are written in terms of conditional branches and phi nodes.

For example the translation of the following uC code:
```
int count(int n) {
  int i = 0;
  while(i < n) {
    i++;
  }
  return i;
}
```
Into LLVM:
```
define i32 @count(i32 %n) {
entry:
   br label %loop
loop:
   %i = phi i32 [ 1, %entry ], [ %nextvar, %loop ]
   %nextvar = add i32 %i, 1
   %cmptmp = icmp ult i32 %i, %n
   %booltmp = zext i1 %cmptmp to i32
   %loopcond = icmp ne i32 %booltmp, 0
   br i1 %loopcond, label %loop, label %afterloop
afterloop:
   ret i32 %i
}
```

## Toolchain

The command line utilities for LLVM can be used to transform IR to and from various forms and run optimizations over it. Everything we can do from the C++ API or llvmpy can also be done from the command line.
```
$ llc example.ll -o example.s             # compile
$ lli example.ll                          # execute
$ opt -S example.bc -o example.ll         # to assembly
$ opt example.ll -o example.bc            # to bitcode
$ opt -O3 example.ll -o example.opt.ll -S # run optimizer
$ opt -view-cfg module.ll                 # view control flow graph
```
And that's basically all you need to know about LLVM.